# 계정별 잔액 조회와 송금 기능이 있는 지갑 사이트
로그인과 지갑, 2개의 화면 및 기능으로 구성됨 - 백엔드 로직을 자세히 알아보자
(1) 로그인 페이지
POST 방식이 아닌 단순 접속 방식으로 접근할 경우 프론트엔드 단계에서 준비될 login.html을 렌더링해 페이지가 나타남
그리고 로그인 버튼이 클릭되면 POST 방식을 통해 백엔드에 접속 
if request.method = 'POST': 입력된 아이디가 input_value에 저장됨
(2) 지갑
블록 정보 조회 URL에 request의 GET 방식으로 접속해서 정보를 받아옴
이후 pandas를 통해 잔액이 조회되고 로그인 계정이 데이터 프레임의 user값과 동일할 경우 해당 잔고 값과 함께 로그인이 성공된 wallet.html 페이지로 렌더링된다

In [ ]:
from flask import Flask
from datetime import datetime
from flask import render_template
from flask import request
from flask import url_for
from flask import redirect

import requests
import json
import os
import pandas as pd

## 로그인
첫 접속시에는 login.html 파일을 랜더링
POST 방식으로 지갑 주소를 입력한 경우,
> 알맞은 지갑주소일 경우 wallet.html 파일 랜더링
> 잘못된 지갑주소일 경우 에러 메시지 띄움

In [ ]:
# Flask app 선언!
app = Flask(__name__, template_folder=os.getcwd())

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method=='POST':
        print("login 버튼을 누름")
        input_value = request.form.to_dict(flat=False) ['wallet_id'][0]
        print("login 지갑주소 : " , input_value)
        ### 기존 user 정보 확인
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        res = requests.get("http://localhost:5000/chain", headers=headers)
        status_json = json.loads(res.text)
        status_json['chain']    
        tx_amount_l = []
        tx_sender_l = []
        tx_reciv_l  = []
        tx_time_l   = []
        # 거래내역 정리 (df_tx)
        for chain_index in range(len(status_json['chain'])):
            chain_tx = status_json['chain'][chain_index]['transactions']
            for each_tx in range(len(chain_tx)):
                tx_amount_l.append(chain_tx[each_tx]['amount'])
                tx_sender_l.append(chain_tx[each_tx]['sender'])
                tx_reciv_l.append(chain_tx[each_tx]['recipient'])
                tx_time_l.append(chain_tx[each_tx]['timestamp'])
        df_tx = pd.DataFrame()
        df_tx['timestamp'] = tx_time_l  
        df_tx['sender'] = tx_sender_l 
        df_tx['recipient'] = tx_reciv_l
        df_tx['amount'] = tx_amount_l   
        df_tx

        # pyBTC 잔고현황 정리 (df_status)
        df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
        df_sended.columns = ['user','sended_amount']
        df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
        df_received.columns = ['user','received_amount']
        df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
        df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
        df_status       
    
        # 결과값 랜더링
        if (df_status['user']==input_value ).sum() == 1:
            print("로그인성공")
            return render_template("wallet.html",  wallet_id = input_value, 
                                                    wallet_value = df_status[df_status['user']== input_value]['balance'].iloc[0])
        else:
            return "잘못된 지갑주소입니다."
        
    return render_template('login.html')

## 지갑 기능
POST 방식으로 보내는 지갑 주소와 보내는 pyBTC 양이 입력된 경우,
해당 지갑주소일 pyBTC 전송 - 송금 URL request의 POST 방식

In [ ]:
@app.route('/wallet', methods=['GET', 'POST'])
def wallet():
    if request.method=='POST':
        send_value = int(request.form.to_dict(flat=False)['send_value'][0] )
        send_target = request.form.to_dict(flat=False)['send_target'][0]
        send_from = request.form.to_dict(flat=False)['send_from'][0]
        print("Login Wallet ID : " ,send_from)
        
        if send_value > 0:
            print("Send Amout :", send_value)
            ## transaction 입력하기
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "sender": send_from,
                "recipient": send_target,
                "amount": send_value,
            }
            requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data))

            return "전송 완료!"

        else:
            return "0 pyBTC 이상 보내주세요!"
        
    return render_template('wallet.html')

## 지갑 사이트 실행
app.run(port=8081)